<a href="https://colab.research.google.com/github/aaronhowellai/machine-learning-projects/blob/main/large%20language%20models/Simple%20ReAct%20LLM%2C%20GPT%20Agent%20From%20Scratch%20using%20OpenAI%20API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🤖 Simple ReAct Agent From Scratch using OpenAI Assistants API**
This notebook follows teaching content from the **Deeplearning.AI** course on [AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/), that encourages its students to code along with the lectures. The following code will be my implementation of the teaching material.

## **✅ The Objective:**
* **Build an agent from scratch**, and understand the division of tasks between the LLM and the code around the LLM.




_______

# **Key Design Patterns of Agentic Workflows** via Andrew Ng
1. Planning
2. Knowing what tools to use
3. Reflection
4. Multi-agent Communication
  * Role playing
5. Memory
  * Tracking results and progress over multiple steps

  * Helpful capabiliies for building agents:
    * Human-in-the-loop input
    * Persistent memory to save states of information, building what is similar to a context window in an LLM chat window

*  **OpenAI Assistants API** (client.beta.threads).
  * Simulate ReAct agent using threads, assistant creation, and message sequencing.

In [17]:
# import packages
import openai
import re
import time

import httpx
from openai import OpenAI
client = OpenAI(
    api_key=" " # redacted for privacy
)

In [11]:
# example response
response = client.responses.create(
    model='gpt-4o-mini',
    input='Write a one-sentence Haiku about Ai Engineering'
)

print(response.output_text)

Lines of code entwined,  
Dreams forged in silicon's glow,  
Future's mind awakens.  


## **Breakdown of Sports Car Weight Assistant**
1. **Agent class**: Handles the loop logic and prompt structure
2. **Actions**: `calculate`, `average_car_weight` as callable Python functions.
3. **Regex Action Detection**
4. **Prompt format**: `assistant.instructions`
5. **Loop**: Driven by `Observation:` inputs and re-running the assistant with each new mesaage

In [28]:
# actions
def calculate(what):
  return str(eval(what))

def average_car_weight(name):
  name = name.lower()
  if "mazda mx-5" in name:
    return "A Mazda MX-5 weighs around 2,400 lbs"
  elif "porsche 911" in name:
    return "A Porsche 911 weighs around 3,400 lbs"
  elif "lamborghini aventador" in name:
    return "A Lamborghini Aventador weighs around 3,500 lbs"
  else:
    return "An average sports car weighs about 3,000 lbs"

known_actions = {
    'calculate': calculate,
    'average_car_weight': average_car_weight
}

# prompt
base_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
Use one or more Action steps to get the information you need - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 3400 + 2400

average_car_weight:
e.g. average_car_weight: Porsche 911

Example session:

Question: How much does a Lamborghini Aventador weigh?
Thought: I should look up the car's weight using average_car_weight
Action: average_car_weight: Lamborghini Aventador
PAUSE

You will be called again with this:
Observation: A Lamborghini Aventador weighs around 3,500 lbs

Then you output:
(When you give the final answer, use this conversational format)
Answer: The Mazda MX-5 (2,400 lbs), Porsche 911 (3,400 lbs), and Lamborghini Aventador (3,500 lbs) have a combined weight of 9,300 lbs.

""".strip()

# ReAct agent class using assistants api
class ReActAgent:
  def __init__(self, prompt=base_prompt, model="gpt-4o-mini"):
    self.model = model
    self.prompt = prompt
    self.assistant = client.beta.assistants.create(
        name="ReActCarWeightAgent",
        instructions=self.prompt,
        model=self.model
    )
    self.thread = client.beta.threads.create()
    self.action_re = re.compile(r"^Action: (\w+): (.*)$")

  def run(self, question, max_turns=5, poll_delay=1):
    print(f"[User] {question}")
    client.beta.threads.messages.create(
        thread_id=self.thread.id,
        role='user',
        content=question
    )

    for turn in range(max_turns):
      run = client.beta.threads.runs.create(
          thread_id=self.thread.id,
          assistant_id=self.assistant.id,
      )

      # poll for completion
      while True:
        time.sleep(poll_delay)
        run_status = client.beta.threads.runs.retrieve(
            thread_id=self.thread.id,
            run_id=run.id
        )
        if run_status.status in {'completed', 'failed'}:
          break

      if run_status.status == 'failed':
        print('[!] Run failed.')
        return

      # get assistant's latest message
      messages = client.beta.threads.messages.list(thread_id=self.thread.id)
      message_content = messages.data[0].content[0].text.value
      print(f'[Assistant] {message_content}')

      # check for action
      matches = self.action_re.findall(message_content)
      if matches:
        for action, arg in matches:
          if action in known_actions:
            result = known_actions[action](arg)
            print(f'[Observation] {result}')
            client.beta.threads.messages.create(
                thread_id=self.thread.id,
                role='user',
                content=f'Observation {result}'
            )
        else:
          print(f'[!] Unknown action: {action}')
          return

      else:
          break

In [25]:
agent = ReActAgent(prompt=base_prompt)
agent.run("What is the combined weight of a Mazda MX-5, a Porsche 911, and a Lamborghini Aventador?")

/tmp/ipython-input-24-2531232644.py:62: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  self.thread = client.beta.threads.create()


[User] What is the combined weight of a Mazda MX-5, a Porsche 911, and a Lamborghini Aventador?


/tmp/ipython-input-24-2531232644.py:67: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
/tmp/ipython-input-24-2531232644.py:74: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/tmp/ipython-input-24-2531232644.py:82: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(
/tmp/ipython-input-24-2531232644.py:94: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=self.thread.id)


[Assistant] Thought: To find the combined weight of these three cars, I will need to retrieve the individual weights of each car first using the average_car_weight action.

Action: average_car_weight: Mazda MX-5  
Action: average_car_weight: Porsche 911  
Action: average_car_weight: Lamborghini Aventador  
PAUSE


In [26]:
agent.run("Observation: A Mazda MX-5 weighs around 2,400 lbs")
agent.run("Observation: A Porsche 911 weighs around 3,400 lbs")
agent.run("Observation: A Lamborghini Aventador weighs around 3,500 lbs")

[User] Observation: A Mazda MX-5 weighs around 2,400 lbs


/tmp/ipython-input-24-2531232644.py:67: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
/tmp/ipython-input-24-2531232644.py:74: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/tmp/ipython-input-24-2531232644.py:82: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(
/tmp/ipython-input-24-2531232644.py:94: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=self.thread.id)


[Assistant] Thought: I have the weight of the Mazda MX-5. Now, I still need the weights of the Porsche 911 and Lamborghini Aventador to proceed with the calculations.

Action: average_car_weight: Porsche 911  
Action: average_car_weight: Lamborghini Aventador  
PAUSE
[User] Observation: A Porsche 911 weighs around 3,400 lbs
[Assistant] Thought: I now have the weights of the Mazda MX-5 and Porsche 911. I still need the weight of the Lamborghini Aventador to find the combined weight of all three cars.

Action: average_car_weight: Lamborghini Aventador  
PAUSE
[User] Observation: A Lamborghini Aventador weighs around 3,500 lbs
[Assistant] Thought: Now that I have the weights of all three cars—Mazda MX-5 (2,400 lbs), Porsche 911 (3,400 lbs), and Lamborghini Aventador (3,500 lbs)—I can calculate their combined weight.

Action: calculate: 2400 + 3400 + 3500  
PAUSE


In [27]:
agent.run("Observation: 9300")

[User] Observation: 9300


/tmp/ipython-input-24-2531232644.py:67: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
/tmp/ipython-input-24-2531232644.py:74: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
/tmp/ipython-input-24-2531232644.py:82: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(
/tmp/ipython-input-24-2531232644.py:94: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=self.thread.id)


[Assistant] Answer: The combined weight of a Mazda MX-5, a Porsche 911, and a Lamborghini Aventador is 9,300 lbs.
